In [7]:
from keras.utils import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

Константы

In [4]:
IMAGE_SIZE = (180, 180)

In [5]:
train_path = '../dataset/chest_xray/train'
val_path = '../dataset/chest_xray/val'
test_path = '../dataset/chest_xray/test'

In [16]:
train_dataset = image_dataset_from_directory(train_path, image_size=IMAGE_SIZE)
test_dataset = image_dataset_from_directory(test_path, image_size=IMAGE_SIZE)
val_dataset = image_dataset_from_directory(val_path, image_size=IMAGE_SIZE)

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


Нормализация датасета

In [17]:
rescale = Rescaling(scale=1.0 / 255)
train_dataset = train_dataset.map(lambda image, label: (rescale(image), label))
val_dataset  = val_dataset.map(lambda image, label: (rescale(image), label))
test_dataset  = test_dataset.map(lambda image, label: (rescale(image), label))